## Importing the libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils

if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor') # To use GPU on pytorch
    device = 'cuda'
else:
    device = 'cpu'

## Preparation of data and weights

In [2]:
# CelebA data: [https://drive.google.com/file/d/1gLviyTH-mXNXD2tq6DO0dnAa-zS0PMk8/view?usp=sharing]
# Setting hyperparameters
batch_size, image_size = 64, 64

# Getting the data and preparing the DataLoader
data_root = './data/processed_celeba_small'
dataset = dset.ImageFolder(root = data_root, 
                              transform = transforms.Compose([
                                  transforms.Resize(image_size),
                                  transforms.CenterCrop(image_size),
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                              ]))

# shuffle = False for fixing "RuntimeError: Expected a 'cuda' device type for generator but found 'cpu'"
data_loader = torch.utils.data.DataLoader(dataset, batch_size = batch_size, shuffle = False, num_workers = 2)

def weights_init(m):
    """
    Takes as input a neural network m 
    and initializes all its weights.
    """
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

## The Generator

In [3]:
class G(nn.Module):
    
    def __init__(self):
        super(G, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(in_channels = 100, out_channels = 512, kernel_size = 4, stride = 1, padding = 0, bias = False),
            nn.BatchNorm2d(num_features = 512),
            nn.ReLU(inplace = True),
            nn.ConvTranspose2d(in_channels = 512, out_channels = 256, kernel_size = 4, stride = 2, padding = 1, bias = False),
            nn.BatchNorm2d(num_features = 256),
            nn.ReLU(inplace = True),
            nn.ConvTranspose2d(in_channels = 256, out_channels = 128, kernel_size = 4, stride = 2, padding = 1, bias = False),
            nn.BatchNorm2d(num_features = 128),
            nn.ReLU(inplace = True),
            nn.ConvTranspose2d(in_channels = 128, out_channels = 64, kernel_size = 4, stride = 2, padding = 1, bias = False),
            nn.BatchNorm2d(num_features = 64),
            nn.ReLU(inplace = True),
            nn.ConvTranspose2d(in_channels = 64, out_channels = 3, kernel_size = 4, stride = 2, padding = 1, bias = False),
            nn.Tanh()
            )
    
    def forward(self, input):
        output = self.main(input)
        return output

In [4]:
netG = G()
netG.apply(weights_init)

G(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

## The Discriminator

In [5]:
class D(nn.Module):
    
    def __init__(self):
        super(D, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size = 4, stride = 2, padding = 1, bias = False),
            nn.LeakyReLU(negative_slope = 0.2, inplace = True),
            nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 4, stride = 2, padding = 1, bias = False),
            nn.BatchNorm2d(num_features = 128),
            nn.LeakyReLU(negative_slope = 0.2, inplace = True),
            nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 4, stride = 2, padding = 1, bias = False),
            nn.BatchNorm2d(num_features = 256),
            nn.LeakyReLU(negative_slope = 0.2, inplace = True),
            nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = 4, stride = 2, padding = 1, bias = False),
            nn.BatchNorm2d(num_features = 512),
            nn.LeakyReLU(negative_slope = 0.2, inplace = True),
            nn.Conv2d(in_channels = 512, out_channels = 1, kernel_size = 4, stride = 1, padding = 0, bias = False),
            nn.Sigmoid()
        )
    
    def forward(self, input):
        output = self.main(input)
        return output.view(-1) # Flatten the output as the same dimension as batch_size

In [6]:
netD = D()
netD.apply(weights_init)

D(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)

## Training the DCGANs

In [ ]:
epochs = 25
criterion = nn.BCELoss()
optimizer_D = optim.Adam(netD.parameters(), lr = 0.0002, betas = (0.5, 0.999))
optimizer_G = optim.Adam(netG.parameters(), lr = 0.0002, betas = (0.5, 0.999))

for epoch in range(epochs):
    for i, data in enumerate(data_loader, 0):
        # 1st Step: Updating the weights of the neural network of the discriminator
        netD.zero_grad()
        
        # Training the discriminator with a real image of the dataset
        real, _ = data
        input = real.to(device = device)
        target = torch.ones(input.size()[0])
        output = netD(input)
        D_error_real = criterion(output, target)
        
        # Training the discriminator with a fake image generated by the generator
        noise = torch.randn(input.size()[0], 100, 1, 1)
        fake = netG(noise)
        target = torch.zeros(input.size()[0])
        output = netD(fake.detach())
        D_error_fake = criterion(output, target)
        
        # Backpropagating the total error
        D_error = D_error_real + D_error_fake
        D_error.backward()
        optimizer_D.step()
        
        # 2nd Step: Updating the weights of the neural network of the generator
        netG.zero_grad()
        target = torch.ones(input.size()[0])
        output = netD(fake)
        G_error = criterion(output, target)
        G_error.backward()
        optimizer_G.step()
        
        # 3rd Step: Printing the losses and saving the real images and the generated images every 100 steps
        print('[%d/%d][%d/%d] Loss_D: %.4f | Loss_G: %.4f' % (epoch, 25, i, len(data_loader), D_error.item(), G_error.item()))
        if i % 100 == 0:
            vutils.save_image(real, '%s/real_samples.png' % "./results_face", normalize = True)
            fake = netG(noise)
            vutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png' % ("./results_face", epoch), normalize = True)